In [1]:
def stat(pred, y):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    if len(pred) == len(y):
        for i in range(len(y)):
            if y[i] == pred[i] and y[i] == 0:
                TN += 1
            elif y[i] == pred[i] and y[i] != 0:
                TP += 1
            elif y[i] != pred[i] and y[i] == 0:
                FP += 1
            else:
                FN += 1
    accuracy = (TN+TP)/(TN+TP+FN+FP)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    
    return TP, TN, FP, FN, accuracy, precision, recall

In [2]:
from tensorflow.keras import models

dnn = models.load_model("dnn")
lstm = models.load_model("lstm")

In [3]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np

df = pd.read_csv("bert_output.csv", encoding="utf-8-sig")
def word2vec(ar, model):
    para_avg_vec = []
    # print("training finished")
    # 对每一段
    for para in ar:
        sum_vec = np.zeros(100)
        # 对每一段中每一个词
        for word in para:
            try:
                # 试着从模型中取得word vector
                sum_vec += model.wv[word]
            except(KeyError):
                # 如果有未记录的特殊符号，就跳过
                continue
        # 对词向量求平均得到句子的向量表示
        para_avg_vec.append(sum_vec / len(para))
    return para_avg_vec

word_2_vec_model = Word2Vec.load('w2v_model')
vec = word2vec(np.array(df["Column6"]), word_2_vec_model)

In [4]:
import tensorflow as tf

x = tf.convert_to_tensor(vec)
dnn_predict = dnn.predict(x)
lstm_predict = lstm.predict(x)
bert_predict = df["Column13"]

y = df["Column7"]

125/125 [==============================] - 1s 8ms/step


In [5]:
def transform(pred, num=[0, 2], num_labels=3):
    delta = num[1] - num[0]
    d = delta / num_labels
    for i in range(len(pred)):
        c = 0
        if pred[i] <= num[0] + d:
            pred[i] = c
        elif pred[i] <= num[0] + 2*d:
            pred[i] = c + 1
        else:
            pred[i] = c + 2
    return pred

dnn_pred, lstm_pred = transform(dnn_predict), transform(lstm_predict)

In [6]:
TP_dnn, TN_dnn, FP_dnn, FN_dnn, accuracy_dnn, precision_dnn, recall_dnn = stat(dnn_pred, y)
accuracy_dnn

0.7525

In [7]:
print(precision_dnn, recall_dnn)

0.8926369241929634 0.7800316957210777


In [8]:
TP_lstm, TN_lstm, FP_lstm, FN_lstm, accuracy_lstm, precision_lstm, recall_lstm = stat(lstm_pred, y)
print(accuracy_lstm, precision_lstm, recall_lstm)

0.6995 0.8398886181691612 0.7648177496038034


In [9]:
TP_bert, TN_bert, FP_bert, FN_bert, accuracy_bert, precision_bert, recall_bert = stat(bert_predict, y)
print(accuracy_bert, precision_bert, recall_bert)

0.49525 0.809368191721133 0.47099841521394614
